### Bedrock LLM 연결
- [langchain package 구조](https://python.langchain.com/docs/versions/v0_3/)
- [aws integration](https://python.langchain.com/docs/integrations/providers/aws/)

In [20]:
from langchain_aws import ChatBedrock
import boto3

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1" 
)

claude_3_client = ChatBedrock(
    client=bedrock_runtime,
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    model_kwargs= { 
    "max_tokens": 512,
    "temperature": 0.0,
},
)

### 챗봇 히스토리 관리 - langchain 0.1 방식

In [21]:
from langchain import LLMChain
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory

template = """
Instruction: 당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.
{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [22]:
llm_chain = LLMChain(
    llm = claude_3_client, 
    prompt=prompt, 
    verbose=True, 
    memory=memory
)

In [23]:
llm_chain.predict(human_input="나의 이름은 제이슨 입니다.")



> Entering new LLMChain chain...
Prompt after formatting:

Instruction: 당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.

Human: 나의 이름은 제이슨 입니다.
Chatbot:

> Finished chain.


'안녕하세요 제이슨님. 반갑습니다. 제가 도와드릴 수 있는 것이 있다면 언제든 말씀해 주세요.'

In [24]:
llm_chain.predict(human_input="내이름이 뭐라고?")



> Entering new LLMChain chain...
Prompt after formatting:

Instruction: 당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.
Human: 나의 이름은 제이슨 입니다.
AI: 안녕하세요 제이슨님. 반갑습니다. 제가 도와드릴 수 있는 것이 있다면 언제든 말씀해 주세요.
Human: 내이름이 뭐라고?
Chatbot:

> Finished chain.


'네, 제가 방금 말씀드린 것처럼 당신의 이름은 제이슨입니다. 제가 잘못 기억하고 있나요?'

In [25]:
memory.load_memory_variables({})

{'chat_history': 'Human: 나의 이름은 제이슨 입니다.\nAI: 안녕하세요 제이슨님. 반갑습니다. 제가 도와드릴 수 있는 것이 있다면 언제든 말씀해 주세요.\nHuman: 내이름이 뭐라고?\nAI: 네, 제가 방금 말씀드린 것처럼 당신의 이름은 제이슨입니다. 제가 잘못 기억하고 있나요?'}

### 챗봇 히스토리 관리 - langchain 0.2 부터는...다른 방식

* Why? 기존 방식의 한계, LLMChain, ConversationChain 등 필요한 Chain을 랭체인에서 만들어서 제공하는 것은 한계가 있다.
* 정형화 되어 커스터마이즈 불가,채팅 서버에서 세션별로 대화내용을 기억하고 싶다면, 랭체인에서 또 Chain 클래스를 만들어서 제공할까?? 귀찮다. 안한다.
* 0.2 부터는 RunnableSequence 를 사용해 필요한 기능을 함수로 커스터마이징 가능하게 제공한다.

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id):
    print(f"[대화 세션ID]: {session_id}")
    if session_id not in store:  
        store[session_id] = ChatMessageHistory()
    return store[session_id]  
    
chat_with_history = RunnableWithMessageHistory(
    runnable=prompt | claude_3_client | StrOutputParser(),
    get_session_history=get_session_history,  
    input_messages_key="human_input",  
    history_messages_key="chat_history", 
)

In [27]:
chat_with_history.invoke(
    {"human_input": "나의 이름은 제이슨 입니다."},
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'안녕하세요 제이슨님, 반갑습니다. 제가 당신의 질문에 대한 답변을 제공해 드리겠습니다. 어떤 질문이 있으신가요?'

In [28]:
chat_with_history.invoke(
    {"human_input": "내이름이 뭐라고?"},
    config={"configurable": {"session_id": "abc123"}},
)


[대화 세션ID]: abc123


'네, 제가 방금 말씀드린 것처럼 당신의 이름은 제이슨입니다. 이를 기억하고 있습니다.'

In [29]:
chat_with_history.invoke(
    {"human_input": "내이름이 뭐라고?"},
    config={"configurable": {"session_id": "abc1234"}},
)

[대화 세션ID]: abc1234


'죄송합니다. 제가 당신의 이름을 알 수 있는 정보가 없습니다. 제가 당신의 이름을 알려드릴 수 없습니다. 대신 제가 도와드릴 수 있는 다른 방법이 있다면 말씀해 주시기 바랍니다.'

In [30]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='나의 이름은 제이슨 입니다.'), AIMessage(content='안녕하세요 제이슨님, 반갑습니다. 제가 당신의 질문에 대한 답변을 제공해 드리겠습니다. 어떤 질문이 있으신가요?'), HumanMessage(content='내이름이 뭐라고?'), AIMessage(content='네, 제가 방금 말씀드린 것처럼 당신의 이름은 제이슨입니다. 이를 기억하고 있습니다.')]),
 'abc1234': InMemoryChatMessageHistory(messages=[HumanMessage(content='내이름이 뭐라고?'), AIMessage(content='죄송합니다. 제가 당신의 이름을 알 수 있는 정보가 없습니다. 제가 당신의 이름을 알려드릴 수 없습니다. 대신 제가 도와드릴 수 있는 다른 방법이 있다면 말씀해 주시기 바랍니다.')])}